In [1]:
# import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import CIFAR10
from PIL import Image
import numpy as np
from scipy import stats
from scipy.stats import wasserstein_distance
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import resize

In [2]:
# function to extract image metadata
def get_image_metadata(image):
    # get width and height of image
    with Image.open(image) as img:
        width, height = img.size

    # calculate image ratio and area
    ratio = width / height
    area = width * height

    # print the metadata
    print(f"image width: {width}")
    print(f"image height: {height}")
    print(f"image ratio: {ratio}")
    print(f"image area: {area}")

#### Image metadata

#### AE Outlier Score

#### KS Drift

#### Data Drift
Wasserstein’s Distance<br>
Kullback — Leibler divergence (KL divergence)<br>
Population Stability Index (PSI)

#### Concept Drift

#### Domain Shift

#### Prediction Drift

#### Upstream Drift